In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
NFL_Kickoffs_2011 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2011.csv')
NFL_Kickoffs_2012 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2012.csv')
NFL_Kickoffs_2013 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2013.csv')
NFL_Kickoffs_2014 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2014.csv')
NFL_Kickoffs_2015 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2015.csv')
NFL_Kickoffs_2016 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2016.csv')
NFL_Kickoffs_2017 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2017.csv')
NFL_Kickoffs_2018 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2018.csv')
NFL_Kickoffs_2019 = pd.read_csv(r'C:\Users\dlbry\Downloads\DSC 640\DSC 640 Final Project\NFL_Kickoffs_2019.csv')

In [2]:
NFL_Kickoffs_2011.head()

,index,Date,Tm,Opp,Quarter,Time,Down,ToGo,Location,Score,Detail,Yds,EPB,EPA,Diff
0,0,9/11/2011,Falcons,Bears,1,10:03,NaN,0.0,CHI 35,Mar-00,"Robbie Gould kicks off 65 yards, touchback",NaN,0.0,0.28,0.28
1,1,9/11/2011,Falcons,Bears,1,1:23,NaN,0.0,CHI 35,3-Oct,"Robbie Gould kicks off 65 yards, touchback",NaN,0.0,0.28,0.28
2,2,9/11/2011,Falcons,Bears,2,8:47,NaN,0.0,CHI 35,13-3,"Robbie Gould kicks off 70 yards, returned by E...",NaN,0.0,-0.22,-0.22
3,3,9/11/2011,Falcons,Bears,2,0:59,NaN,0.0,CHI 35,16-3,"Robbie Gould kicks off 65 yards, touchback",NaN,0.0,0.28,0.28
4,4,9/11/2011,Falcons,Bears,3,15:00,NaN,0.0,CHI 35,16-Mar,"Robbie Gould kicks off 66 yards, returned by E...",NaN,0.0,-0.38,-0.38


NFL kickoff datasets used in this project were downloaded from [Stathead Football](https://stathead.com/football/play_finder.cgi). After downloading the yearly kickoff stats, I wanted to determine the percentage of normal kicks (i.e. kicks from the 35 yard-line) that did not reach the endzone (i.e. they were kicked less than 65 yards). To do this, I first filtered the Location column for rows containing 35, since those are normal kickoff spots. Next, I split the Detail string and kept everything to the left of the comma, and then filtered out everything but numbers, which gives the distance of the kick. Rows with missing/na Locations or Details were dropped. To calculate the percentage of normal kicks that didn't reach the endzone, I created a filter to cound the number of kicks less than 65 yards that were also kicked from the 35 yard line. Then I divided that number by the total number of normal kicks. These steps take place in the function below, and the percentage is returned when a dataframe is passed into the function. 

In [3]:
def calc_pct_of_kicks_short_of_endzone(df):
    
    # Create a filter for normal kickoffs from the 35 yardline
    normal_kickoff_mask = df['Location'].str.contains('35')
    
    # Drop na values
    df.dropna(subset=['Location'],inplace=True)
        
    # Split the Detail column to determine kickoff distance
    before_comma = df['Detail'].str.split(',').str[0]
    
    # Return only the number from the string, which is the kickoff distance
    df['kickoff_distance'] = before_comma.str.replace(r'\D+', '')
    
    # Remove kickoffs with blank distances
    indexNames = df[df['kickoff_distance']==''].index
    # Delete these row indexes from dataFrame
    df.drop(indexNames , inplace=True)
    
    # Convert to integer
    df['kickoff_distance'] = df['kickoff_distance'].astype(str).astype(int)
    
    # Create a filter for kickoffs that don't reach the endzone
    kicks_before_endzone_mask = df['kickoff_distance']<65
    
    # Calculate the number of normal kicks that don't reach the endzone
    num_of_kicks_before_endzone = len(df[normal_kickoff_mask&kicks_before_endzone_mask])
    
    # Calculate the number total number normal kicks
    total_num_of_kicks = len(df[normal_kickoff_mask])
    
    # Calculate the % of normal kicks that didn't reach the endzone
    pct_of_kicks_before_endzone = round((num_of_kicks_before_endzone/total_num_of_kicks)*100,2)
    
    # Return a dataframe of kickoffs that 
    return pct_of_kicks_before_endzone

We will create a list of dataframes to pass into the functions, and a separate list of matching years. We will use the list of dataframes to create a list of the percentage of kicks that were short of the endzone. We will finally match the years up with the percentages and create a dataframe of the two columns of values. 

In [4]:
# Create a list of the dataframes
list_of_df = [NFL_Kickoffs_2011,
             NFL_Kickoffs_2012,
             NFL_Kickoffs_2013,
             NFL_Kickoffs_2014,
             NFL_Kickoffs_2015,
             NFL_Kickoffs_2016,
             NFL_Kickoffs_2017,
             NFL_Kickoffs_2018,
             NFL_Kickoffs_2019]

# Create a list of years
list_of_years = list(range(2011,2020))

In [5]:
# Create a list of the percent of short kicks for each year
list_of_pcts = [calc_pct_of_kicks_short_of_endzone(x) for x in list_of_df]

In [6]:
# Calling DataFrame constructor after zipping 
# the list of years and list of percents, and specifying column names
kickoffs_not_reaching_endzone = pd.DataFrame(list(zip(list_of_years, list_of_pcts)),
               columns =['Year', 'kickoff_pct_not_reaching_the_end_zone'])



kickoffs_not_reaching_endzone

,Year,kickoff_pct_not_reaching_the_end_zone
0,2011,17.84
1,2012,19.08
2,2013,18.41
3,2014,14.94
4,2015,13.12
5,2016,22.32
6,2017,23.23
7,2018,22.56
8,2019,22.15
